In [23]:
import cv2
import cv2 as cv
import numpy as np
import pandas as pd
import glob
import os
import pickle
from scipy import ndimage as nd
from matplotlib import pyplot as plt

In [50]:
# 이미지 경로
img_path = '../BoneAge/Data/image/'
data_path = '../BoneAge/Data/'
filter_path = '../BoneAge/Data/filtering_img/'

original_img_path = glob.glob(img_path + "*.jpg")
filtering_img = os.path.join(data_path, 'filtering_img')
os.makedirs(filtering_img, exist_ok=True)

filtering_path = glob.glob(filter_path + "*.jpg")
masking_img = os.path.join(data_path, "masking_img")
os.makedirs(masking_img, exist_ok=True)

## 이미지 불러오기

In [51]:
# 이미지 불러오기 및 이미지 사이즈 재설정
def read_img(path):
    original_img = plt.imread(path, cv2.IMREAD_GRAYSCALE)
    return original_img

## 이미지 전처리

In [44]:
# 이미지 사이즈 재설정, 정규화, 노이즈 제거, 평탄화 진행
def filtering(original_img):
    img = original_img.copy()

    # 사이즈 재설정
    resized_img = cv2.resize(img, (800, 1000))

    # 정규화
    normalized_image = cv2.normalize(resized_img, None, 0, 255, cv2.NORM_MINMAX)

    # 노이즈 제거
    denoised_image = cv2.fastNlMeansDenoising(normalized_image, None, h=3, templateWindowSize=21, searchWindowSize=30)
    
    # 평탄화
    gray_image = cv2.cvtColor(denoised_image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    equalized_image = clahe.apply(gray_image)
    
    return equalized_image

## 전체이미지 filtering 실행

In [27]:
# 원본 이미지 전체 전처리 진행
for img_path in original_img_path:
    original_img = read_img(img_path)
    filtered_img = filtering(original_img)
    
    # 이미지 파일명 추출
    img_filename = os.path.basename(img_path)
    
    # 저장할 경로 설정
    save_path = os.path.join(filtering_img, img_filename)
    
    # 이미지 저장
    cv2.imwrite(save_path, filtered_img)

## 마스크 생성

In [59]:
# 마스크 생성
def get_mask(img):
    # 밝기를 강조한 Lab으로 이미지 변환
    img_mask = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)

    # blur처리
    # 이미지 평균 값을 기준으로 홀수값 만들기
    blur_k = int((img_mask.mean()*0.5)//2)*2+1 
    img_mask = cv2.medianBlur(img_mask, blur_k)

    # threshold 적용을 위해 Lab에서 Grayscale로 이미지 변환
    img_mask = cv2.cvtColor(img_mask, cv2.COLOR_Lab2BGR)
    img_mask = cv2.cvtColor(img_mask, cv2.COLOR_BGR2GRAY)

    # 평균값을 기준으로 이진화
    ret, img_mask = cv2.threshold(img_mask, img_mask.mean()*0.94, 225, cv2.THRESH_BINARY)

    # Max value of contours for make mask
    contours, hierarchy = cv2.findContours(img_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_cnt = max(contours, key=cv2.contourArea)
    mask = np.zeros(img_mask.shape, dtype=np.uint8)
    cv2.drawContours(mask, [max_cnt], -1, (255,255,255), -1)

    # dilate 적용
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    mask = cv2.dilate(mask,k)
    
    return mask


In [60]:
# 마스크를 이용하여 배경 자르기
def cut_mask(img, mask):
    img_for_cut = img.copy()
    height, width = img_for_cut.shape[:2]

    # mask
    mask_list = mask.tolist()
    
    for y in range(int(height*0.05),height):
        if max(mask[y,int(width*0.3):int(width*0.7)]) > 0:
            start_y = y-int(height*0.05)
            break
            
    for x in range(int(width*0.05),width):
        if max(mask[int(height*0.3):int(height*0.7),x]) > 0:
            start_x = x-int(width*0.05)
            break
            
    for x in range(int(width*0.95),-1,-1):
        if max(mask[int(height*0.3):int(height*0.7),x]) > 0:
            end_x = x+int(width*0.05)
            break
            
    cut_index = 0
    if mask_list[height-1][-1] == 255 or mask_list[height-1][0] == 255:
        for n in reversed(range(height)):
            if mask_list[n][0] == 0 or mask_list[n][-1] == 0:
                cut_index = n
                break
                
    if cut_index == 0:
        cut_index = height

    # converting color
    img_for_cut = cv2.cvtColor(img_for_cut, cv2.COLOR_BGR2GRAY) 

    img_for_cut = img_for_cut[start_y:(cut_index-1),start_x:end_x]
    mask = mask[start_y:(cut_index-1),start_x:end_x]

    # remove background
    masked = cv2.bitwise_and(img_for_cut, mask)

    return masked